In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Create Fully Connected Networks

In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN,self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)


  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)

    return x

In [ ]:
model = NN(784, 10)
x =torch.randn(64, 784)
model(x).shape

torch.Size([64, 10])

# Set Device

In [ ]:
device= torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

# HyperParameters

In [ ]:
input_size= 784
num_classes = 10
learning_rate= 0.001
batch_size = 64
num_epochs = 1

# Load Data

In [ ]:
train_dataset = datasets.MNIST(root='datasets/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='datasets/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 117153290.65it/s]


Extracting datasets/MNIST/raw/train-images-idx3-ubyte.gz to datasets/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 70591896.17it/s]

Extracting datasets/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 31224677.62it/s]


Extracting datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23346236.24it/s]


Extracting datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/MNIST/raw



# Initialize Network

In [ ]:
model = NN(input_size= input_size, num_classes=num_classes).to(device)

# Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Train Network

In [ ]:
for epoch in range(num_epochs):
  for data_idx, (data, target) in enumerate(train_loader):

    # get data to cuda if possible
    data = data.to(device = device)
    target = target.to(device = device)

    # get to correct shape
    data = data.reshape(data.shape[0], -1)

    # forward
    scores = model(data)
    loss = criterion(scores, target)

    #backward
    optimizer.zero_grad() # Essentially set all the gradients to zero for each batch
    loss.backward()

    # gradient descent or adam step
    optimizer.step()

# Chech Accuracy on training and testing to see how good our model

In [ ]:
def check_accuracy(loader, model):

  if loader.dataset.train:
    print('Checking Accuracy of train data: ')
  else:
    print('Checking Accuracy of test data: ')

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    # We don't have to compute the gradient that would be unnessessory computing
    for x, y in loader:
      x = x.to(device= device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
  model.train()


In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Checking Accuracy of train data: 
Got 55782 / 60000 with accuracy 92.97
Checking Accuracy of test data: 
Got 9284 / 10000 with accuracy 92.84
